### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk

In [2]:
import json
import nltk
import numpy as np

In [3]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [4]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [5]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    #print(summary_str)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    texts = []
    for i,w in enumerate(case_content):
        label = getLabelWithWordIndex(i,ne_index_list)
        texts.append((w,label))
        
    docs.append(texts)

print(t)
print(c)

13452
0


In [8]:
for w in docs[0]:
    print(w[0],w[1])

李 O
某 O
， O
女 O
， O
6 O
7 O
岁 O
。 O
每 O
于 O
夏 O
日 O
洗 O
澡 O
后 O
即 O
寒 B-S
热 I-S
不 I-S
适 E-S
， O
自 O
感 O
发 O
热 O
而 O
又 O
需 O
披 O
衣 O
裹 O
护 O
。 O
并 O
于 O
不 O
知 O
不 O
觉 O
中 O
汗 B-S
透 I-S
衣 I-S
服 E-S
， O
急 O
又 O
换 O
衣 O
， O
如 O
此 O
三 O
四 O
次 O
方 O
渐 O
趋 O
平 O
复 O
。 O
年 O
年 O
如 O
此 O
， O
以 O
至 O
夏 O
日 O
视 O
洗 O
澡 O
为 O
畏 O
途 O
。 O
曾 O
住 O
院 O
治 O
疗 O
不 O
效 O
。 O
中 O
医 O
鉴 O
于 O
盛 O
夏 O
炎 O
热 O
， O
每 O
以 O
涤 O
暑 O
益 O
气 O
， O
敛 O
汗 O
固 O
表 O
为 O
治 O
， O
总 O
不 O
见 O
效 O
。 O
三 O
年 O
前 O
来 O
我 O
处 O
就 O
诊 O
。 O
症 O
如 O
前 O
述 O
， O
脉 O
虚 O
大 O
而 O
迟 O
， O
苔 O
薄 O
黄 O
。 O
乃 O
处 O
桂 O
枝 O
汤 O
加 O
黄 O
芪 O
、 O
炮 O
附 O
片 O
， O
1 O
剂 O
效 O
， O
2 O
剂 O
愈 O
。 O
后 O
每 O
年 O
夏 O
日 O
发 O
作 O
， O
均 O
来 O
服 O
药 O
2 O
剂 O
即 O
愈 O
。 O


In [5]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 2701
print(ave)

606.4124398370974


In [6]:
print(len(docs))

2107


In [7]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

print(data[0])

[('李', 'JJ', 'O'), ('某', 'NNP', 'O'), ('，', 'NNP', 'O'), ('女', 'NNP', 'O'), ('，', 'VBD', 'O'), ('6', 'CD', 'O'), ('7', 'CD', 'O'), ('岁', 'NN', 'O'), ('。', 'NNP', 'O'), ('每', 'NNP', 'O'), ('于', 'NNP', 'O'), ('夏', 'NNP', 'O'), ('日', 'NNP', 'O'), ('洗', 'NNP', 'O'), ('澡', 'NNP', 'O'), ('后', 'NNP', 'O'), ('即', 'NNP', 'O'), ('寒', 'NNP', 'B-S'), ('热', 'NNP', 'I-S'), ('不', 'NNP', 'I-S'), ('适', 'NNP', 'E-S'), ('，', 'NNP', 'O'), ('自', 'NNP', 'O'), ('感', 'NNP', 'O'), ('发', 'NNP', 'O'), ('热', 'NNP', 'O'), ('而', 'NNP', 'O'), ('又', 'NNP', 'O'), ('需', 'NNP', 'O'), ('披', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('裹', 'NNP', 'O'), ('护', 'NNP', 'O'), ('。', 'NNP', 'O'), ('并', 'NNP', 'O'), ('于', 'NNP', 'O'), ('不', 'NNP', 'O'), ('知', 'NNP', 'O'), ('不', 'NNP', 'O'), ('觉', 'NNP', 'O'), ('中', 'NNP', 'O'), ('汗', 'NNP', 'B-S'), ('透', 'NNP', 'I-S'), ('衣', 'NNP', 'I-S'), ('服', 'NNP', 'E-S'), ('，', 'NNP', 'O'), ('急', 'NNP', 'O'), ('又', 'NNP', 'O'), ('换', 'NNP', 'O'), ('衣', 'NNP', 'O'), ('，', 'NNP', 'O'), ('如', 'NNP', 'O'),

In [8]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [9]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
print(len(X_train[0][4]))
print(X_train[0][5])

18
['bias', 'word.lower=，', 'word[-3:]=，', 'word[-2:]=，', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=VBD', '-1:word.lower=性', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NNP', '+1:word.lower=4', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=True', '+1:postag=CD']


In [11]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 29999
Seconds required: 2.952

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 205362.721259
Feature norm: 1.000000
Error norm: 101122.583985
Active features: 29858
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.073

***** Iteration #2 *****
Loss: 202867.983675
Feature norm: 1.019942
Error norm: 83837.390959
Active features: 28749
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.552

***** Iteration #3 *****
Loss: 194457.331824
Feature norm: 1.112004
Error norm: 50843.970699
Active features: 22311
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #40 *****
Loss: 41482.135736
Feature norm: 20.391698
Error norm: 34122.276918
Active features: 25446
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #41 *****
Loss: 40767.832771
Feature norm: 20.782122
Error norm: 33469.241670
Active features: 25364
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #42 *****
Loss: 40143.773065
Feature norm: 21.170862
Error norm: 27802.779638
Active features: 25300
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #43 *****
Loss: 39345.134477
Feature norm: 21.724338
Error norm: 27879.738597
Active features: 25204
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #44 *****
Loss: 38665.526644
Feature norm: 22.402592
Error norm: 25125.576785
Active features: 25117
Line search trials: 1
Line search step: 1.0

***** Iteration #82 *****
Loss: 26187.481104
Feature norm: 51.844385
Error norm: 1318.016604
Active features: 21834
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #83 *****
Loss: 26131.813699
Feature norm: 52.249025
Error norm: 2728.516101
Active features: 21884
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #84 *****
Loss: 26011.873266
Feature norm: 54.520709
Error norm: 13453.853327
Active features: 21685
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #85 *****
Loss: 25938.741161
Feature norm: 54.590057
Error norm: 2361.911460
Active features: 21749
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #86 *****
Loss: 25897.091208
Feature norm: 54.977363
Error norm: 1739.086284
Active features: 21738
Line search trials: 1
Line search step: 1.00000

***** Iteration #122 *****
Loss: 24846.157650
Feature norm: 66.908362
Error norm: 6608.856231
Active features: 20835
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.555

***** Iteration #123 *****
Loss: 24832.926921
Feature norm: 66.984144
Error norm: 3284.342593
Active features: 20863
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #124 *****
Loss: 24825.176414
Feature norm: 67.130385
Error norm: 2696.335485
Active features: 20834
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.545

***** Iteration #125 *****
Loss: 24818.734908
Feature norm: 67.181406
Error norm: 1869.439693
Active features: 20848
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #126 *****
Loss: 24812.337562
Feature norm: 67.334573
Error norm: 3651.004370
Active features: 20821
Line search trials: 1
Line search step: 1.0

***** Iteration #163 *****
Loss: 24674.781233
Feature norm: 70.637098
Error norm: 1736.602984
Active features: 20558
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.554

***** Iteration #164 *****
Loss: 24673.952783
Feature norm: 70.714973
Error norm: 2894.379294
Active features: 20570
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #165 *****
Loss: 24670.785677
Feature norm: 70.750139
Error norm: 1623.265331
Active features: 20562
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #166 *****
Loss: 24670.077961
Feature norm: 70.824944
Error norm: 2883.619666
Active features: 20546
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.551

***** Iteration #167 *****
Loss: 24666.816862
Feature norm: 70.861473
Error norm: 1485.980811
Active features: 20545
Line search trials: 1
Line search step: 1.0

In [12]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred0 = [tagger.tag(xseq0) for xseq0 in X_train]
y_pred = [tagger.tag(xseq) for xseq in X_test]


In [13]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions0 = np.array([labels[tag] for row in y_pred0 for tag in row])
truths0 = np.array([labels[tag] for row in y_train for tag in row])

# Print out the classification report
print(classification_report(
    truths0, predictions0,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

#print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          O       0.98      1.00      0.99   1266487
      B-SYM       0.59      0.21      0.32      9811
      I-SYM       0.61      0.26      0.36     15193
      E-SYM       0.60      0.22      0.32      9785

avg / total       0.97      0.98      0.97   1301276



In [14]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.98      0.99      0.99    327510
      B-SYM       0.48      0.18      0.26      2541
      I-SYM       0.41      0.18      0.25      4061
      E-SYM       0.49      0.18      0.27      2532

avg / total       0.96      0.97      0.97    336644

